Webスクレイピングに役立つモジュール

|モジュール|説明|
|:-|:-|
|webbrowser|Python付属のモジュール。指定したページをブラウザで開く。|
|Requests|インターネットからファイルやWebページをダウンロードする。|
|Beautiful Soup|Webページの記法であるHTMLをパース(文法的に解釈)する。|
|Selenium|Webブラウザを起動し制御する。Seleniumはブラウザ上のフォームを記入したり、マウスクリックをシミュレートできる。|

# webbrowserモジュール

In [1]:
import webbrowser

webbrowser.open("http://inventwithpython.com/")

True

webbrowser をインポートして webbrowser.open 関数にURLを渡すことでWebブラウザを使って開いてくれる

webbrowser ができるのはこれだけである

# requestsモジュール

In [2]:
import requests

res = requests.get("https://automatetheboringstuff.com/files/rj.txt")

In [3]:
type(res)

requests.models.Response

In [4]:
res.status_code == requests.codes.ok

True

In [5]:
len(res.text)

178978

In [6]:
print(res.text[: 250])

The Project Gutenberg EBook of Romeo and Juliet, by William Shakespeare

This eBook is for the use of anyone anywhere at no cost and with
almost no restrictions whatsoever.  You may copy it, give it away or
re-use it under the terms of the Projec


requests は外部モジュールなのでインストール事前にする必要がある

requests をインポートし、requests.get関数にダウンロードするURLを渡すことで Response オブジェクトを返す

Webページのリクエストが成功したかどうかはResponseオブジェクトの status_code アトリビュートを調べればわかる

status_code の値が requests.codes.ok と同じ(200)であれば成功している(Not Foundは404)

ダウンロードしたWebページはResponseオブジェクトの text アトリビュートに入っていて、呼び出すことで表示することができる

In [7]:
res = requests.get("http://inventwithpython.com/page_that_does_not_exist")

res.raise_for_status()

HTTPError: 404 Client Error: Not Found for url: http://inventwithpython.com/page_that_does_not_exist

Responseオブジェクトの raise_for_status メソッドを使うとファイルのダウンロードを失敗するとエラーを起こす

エラーを起こしてプログラムを停止させたくないときはtry/except文を使う

In [8]:
res = requests.get("https://automatetheboringstuff.com/files/rj.txt")
res.raise_for_status()
with open(r".\pydata\RomeoAndJuliet.txt", "wb") as play_file:
    for chunk in res.iter_content(1000000):
        play_file.write(chunk)

requests のResponseオブジェクトを作ってダウンロードしたら、通常の open 関数と write メソッドを使うことでファイルをPCに保存することができる

しかし、Web上のテキストはバイト型のため write メソッドの第二引数に渡すモードは wb を使わなくてはいけない

Responseオブジェクトの iter_content メソッドは渡した数値(上では100kb)ごとのデータのチャンク(塊)を返してくれる

つまり100kbずつ write メソッドを使って書き出してくれていることになる

# BeautifulSoupモジュール(bs4)

In [9]:
import requests, bs4
res = requests.get("http://nostarch.com")
res.raise_for_status()
no_starch_soup = bs4.BeautifulSoup(res.text)
type(no_starch_soup)

bs4.BeautifulSoup

bs4 の BeautifulSoup 関数にResponseオブジェクトのtextメソッド(解析するHTML)を渡すことで BesutifulSoupオブジェクトを作ることができる

In [10]:
example_file = open(r".\pyworks\example.html")
example_soup = bs4.BeautifulSoup(example_file)
type(example_soup)

bs4.BeautifulSoup

他にもHTMLのファイルオブジェクトを渡すことでもBeautifulSoupオブジェクトを作ることもできる

In [11]:
example_file = open(r".\pyworks\example.html")
example_soup = bs4.BeautifulSoup(example_file)
elems = example_soup.select("#author")
type(elems)

bs4.element.ResultSet

In [12]:
len(elems)

1

In [13]:
type(elems[0])

bs4.element.Tag

In [14]:
elems[0].getText()

'Al Sweigart'

In [15]:
str(elems[0])

'<span id="author">Al Sweigart</span>'

In [16]:
elems[0].attrs

{'id': 'author'}

BeautifulSoupオブジェクトの select メソッドにCSSセレクタ(正規表現のようなもの)を渡すことでWebページの要素を取得できる

よく使われるCSSクラスタの文法

|select()に渡すセレクタ|マッチする対象|
|:-|:-|
|soup.select("div")|すべての`<div>`要素|
|soup.select("#author")|id属性がauthorである要素|
|soup.select(".notice")|CSSクラス属性がnoticeである全要素|
|soup.select("div span")|`<div>`要素の中のすべての`<span>`要素|
|soup.select("div > span")|`<div>`要素の直下のすべての`<span>`要素(間に他の要素がない)|
|soup.select("input[name]")|name属性(値は任意)を持つすべての`<input>`要素|
|soup.select("input[type='button']")|type属性の値がbuttonであるすべての`<input>`要素|

select メソッドにBeautifulSoupオブジェクトを渡すことでTagオブジェクトのリストのようなもの(Resultsetオブジェクト)を返し、リストの中にはマッチしたすべての要素が入っている

Tagオブジェクトの getText メソッドを使うと、開始タグと終了タグを除いた要素の内部テキストを取得できる

Tagオブジェクトをstr関数に渡すと、開始タグと終了タグを含んだ文字列を返す

Tagオブジェクトのattrsアトリビュートを呼ぶと、要素の属性を辞書として保持しているのを返してくれる

In [17]:
p_elems = example_soup.select("p")
len(p_elems)

3

In [18]:
str(p_elems[0])

'<p>Download my <strong>Python</strong> book from <a href="http://inventwithpython.com">my website</a>.</p>'

In [19]:
p_elems[0].getText()

'Download my Python book from my website.'

In [20]:
str(p_elems[1])

'<p class="slogan">Learn Python the easy way!</p>'

In [21]:
p_elems[1].getText()

'Learn Python the easy way!'

In [22]:
str(p_elems[2])

'<p>By <span id="author">Al Sweigart</span></p>'

In [23]:
p_elems[2].getText()

'By Al Sweigart'

BeautifulSoupオブジェクトからすべての`<p>`要素を取り出すこともできる

In [24]:
soup = bs4.BeautifulSoup(open(r".\pyworks\example.html"))
span_elem = soup.select("span")[0]
str(span_elem)

'<span id="author">Al Sweigart</span>'

In [25]:
span_elem.get("id")

'author'

In [26]:
span_elem.get("some_noneexistent_addr") == None

True

In [27]:
span_elem.attrs

{'id': 'author'}

Tagオブジェクトの get メソッドに属性名を渡すと属性値を返してくれる

# Seleniumモジュール

In [28]:
from selenium import webdriver
browser = webdriver.Firefox()
type(browser)

selenium.webdriver.firefox.webdriver.WebDriver

In [29]:
browser.get("http://inventwithpython.com")

selenium から webdriver をインポートして、webdriverの Firefox 関数を使うとFirefoxのWebブラウザが起動し、その時に返すWebdriver型のオブジェクトを使って操作ができる

Webdriver型のメソッドである get メソッドを使ってURLを開くことができる

In [30]:
browser = webdriver.Firefox()
browser.get("http://inventwithpython.com")

try:
    elem = browser.find_element_by_class_name("bookcover")
    print(f"そのクラス名を持つ要素 <{elem.tag_name}> を見つけた")
except:
    print("そのクラス名を持つ要素は見つからなかった")

そのクラス名を持つ要素は見つからなかった


Webdriverオブジェクトにはページの要素を見つけるためのメソッドがいくつかあり、大きく分けて find_element_* と find_elements_* の2種類がある

find_element_* メソッドは検索条件にマッチした最初の要素を WebElementオブジェクトとして返す

find_elements_* メソッドは検索条件にマッチしたすべての要素を WebElementオブジェクト リストとして返す

要素を検索するSeleniumのWebDriverメソッド一覧

|メソッド名|返されるWebElementオブジェクトまたはそのリスト|
|:-|:-|
|browser.find_element_by_class_name(name)|CSSクラスがnameである要素のオブジェクト|
|browser.find_elements_by_class_name(name)|CSSクラスがnameである要素のリスト|
|browser.find_element_by_css_selector(selector)|CSSセレクタにマッチする要素のオブジェクト|
|browser.find_elements_by_css_selector(selector)|CSSセレクタにマッチする要素のリスト|
|browser.find_element_by_id(id)|id属性が一致する要素のオブジェクト|
|browser.find_elements_by_id(id)|id属性が一致する要素のリスト|
|browser.find_element_by_link_text(text)|textに完全一致する`<a>`要素のオブジェクト|
|browser.find_elements_by_link_text(text)|textに完全一致する`<a>`要素のリスト|
|browser.find_element_by_partial_link_text(text)|textに部分一致する`<a>`要素のオブジェクト|
|browser.find_elements_by_partial_link_text(text)|textに部分一致する`<a>`要素のリスト|
|browser.find_element_by_name(name)|name属性が一致する要素のオブジェクト|
|browser.find_elements_by_name(name)|name属性が一致する要素のリスト|
|browser.find_element_by_tag_name(name)|タグ名がnameに一致する要素のオブジェクト(大文字と小文字を区別しない)|
|browser.find_elements_by_tag_name(name)|タグ名がnameに一致する要素のリスト(大文字と小文字を区別しない)|

WebElementオブジェクトに使えるアトリビュートとメソッド一覧

|属性もしくはメソッド|説明|
|:-|:-|
|tag_name|タグ名。`<a>`なら"a"|
|get_attribute(name)|要素の属性nameの値|
|text|要素の内部テキスト。`<spam>hello</spam>`なら"hello"|
|clear()|inputやtextarea要素のテキストを消去する|
|is_displayed()|要素が可視ならTrue、そうでなければFalseを返す|
|is_enabled()|input要素が有効ならTrue、無効ならFalseを返す|
|is_selected()|checkboxやradiobutton要素がチェックされていればTrue、されていなければFalseを返す|
|location|"x"と"y"のキーを持つ辞書で、要素のページ上の座標を示す|

In [36]:
browser = webdriver.Firefox()
browser.get("http://inventwithpython.com")
link_elem = browser.find_element_by_link_text("See what's new in the second edition.")
type(link_elem)

selenium.webdriver.firefox.webelement.FirefoxWebElement

In [37]:
link_elem.click()

WebElementオブジェクトには click というメソッドがあり、その要素上でクリックをすることができる

In [ ]:
browser = webdriver.Firefox()
browser.get("https://mail.yahoo.com") # 現在はアドレスが変更されている
email_elem = browser.find_element_by_id("login-username")
email_elem.send_keys("not_my_real_email") # メールアドレスを内容を入力欄に記入
password_elem = browser.find_element_by_id("login-passwd")
password_elem.send_keys("12345")　# パスワードを入力欄に記入
password_elem.submit() # 入れた内容を送信

Webページ上のテキスト入力欄にキー入力するには`<input>`や`<textarea>`要素を見つけ、WebElementオブジェクトに send_keys メソッドを使い、入力したい文字列を渡すことで入力できる

そして、WebElementオブジェクトに submit メソッドを呼び出すことで送信をすることができる

In [40]:
from selenium.webdriver.common.keys import Keys
browser = webdriver.Firefox()
browser.get("http://nostarch.com")
html_elem = browser.find_element_by_tag_name("html")
html_elem.send_keys(Keys.END) # 末尾にスクロール
html_elem.send_keys(Keys.HOME) # 先頭にスクロール

selenium.webdriver.common.keys メソッドの Keys アトリビュート(長いので`from selenium.webdriver.common.keys import Keys`でインポート)をsend_keys メソッドに渡すことで特殊なキーを入力することができる

キーの入力を送るのは`<html>`タグが適切(HTMLファイルは`<html>`で囲まれているため)

selenium.webdriver.common.keysモジュールでよく使われる値一覧

|アトリビュート|意味|
|:-|:-|
|Keys.DOWN ,Keys.UP ,Keys.LEFT ,Keys.RIGHT|矢印キー|
|Keys.ENTER ,Keys.RETURN|Enter, Returnキー|
|Keys.HOME ,Keys.END ,Keys.PAGE_DOWN ,Keys.PAGE_UP|Home ,End ,PageDown ,PageUpキー|
|Keys.ESCAPE ,Keys.BACK_SPACE ,Keys.DELETE|Esc ,Backspace ,Deleteキー|
|Keys.F1 ,... ,Keys.F12|F1~F12キー|
|Keys.TAB|Tabキー|

In [ ]:
browser.quit() # ウィンドウを閉じるをクリックする

ウィンドウを閉じるボタンをクリックするにはWebdriver型オブジェクトの quit メソッド呼ぶことでできる

ブラウザのボタンをクリックするメソッド一覧

|Webdriver型オブジェクトのメソッド|説明|
|:-|:-|
|browser.back()|戻るボタンをクリックする|
|browser.forward()|進むボタンをクリックする|
|browser.refresh()|再読み込みボタンをクリックする|
|browser.quit()|ウィンドウを閉じるボタンをクリックする|